<a href="https://colab.research.google.com/github/ashimasinghal/Indian-NER/blob/main/telegu_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seqeval
!pip install transformers

In [ ]:

import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

In [ ]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [ ]:
!pip list | grep -E 'transformers|torch|Keras'

Keras-Preprocessing           1.1.2
torch                         1.10.0+cu111
torchaudio                    0.10.0+cu111
torchsummary                  1.5.1
torchtext                     0.11.0
torchvision                   0.11.1+cu111
transformers                  4.12.5


In [ ]:
# load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BERT/NER Corpus/NER_data.txt', delim_whitespace=True, header=None )


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df

,0,1,2,3
0,ఢిల్లీ,DhillI,B-LOC,NNP
1,పీఠంపై,pITha~mpai,O,NN
2,కేజ్రీవాల్‌,kEjrIvAl‌,B-PERSON,NNP
3,కన్ను,kannu,O,NN
4,పడింది,paDia~mdi,O,VM
...,...,...,...,...
47961,మిల్లర్,millar,I-PERSON,NN
47962,10,10,B-NUM,QC
47963,నాటౌట్,nATauT,O,NN
47964,),),O,SYM


In [ ]:
sentence_id = []
words = df[0]
sentence = 1
for word in words:
  sentence_id.append(sentence)
  if(word=='.'):
    sentence += 1

In [ ]:
df.insert(1,'Sentence ID',sentence_id,allow_duplicates=True)
df.head(50)

,0,Sentence ID,1,2,3
0,ఢిల్లీ,1,DhillI,B-LOC,NNP
1,పీఠంపై,1,pITha~mpai,O,NN
2,కేజ్రీవాల్‌,1,kEjrIvAl‌,B-PERSON,NNP
3,కన్ను,1,kannu,O,NN
4,పడింది,1,paDia~mdi,O,VM
5,.,1,.,O,SYM
6,ముఖ్యమంత్రి,2,mukhyama~mtri,O,NN
7,పదవిలో,2,padavilO,O,NN
8,ముచ్చటగా,2,muccaTagA,O,RB
9,మూడు,2,mUDhu,B-PERIOD,QC


In [ ]:
hindi = pd.DataFrame()
hindi = df[['Sentence ID',0,3,2]]
hindi.rename(columns={'Sentence ID': 'Sentence', 0: 'word', 3: 'POS', 2:'TAG'}, inplace=True)
hindi

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Sentence,word,POS,TAG
0,1,ఢిల్లీ,NNP,B-LOC
1,1,పీఠంపై,NN,O
2,1,కేజ్రీవాల్‌,NNP,B-PERSON
3,1,కన్ను,NN,O
4,1,పడింది,VM,O
...,...,...,...,...
47961,3157,మిల్లర్,NN,I-PERSON
47962,3157,10,QC,B-NUM
47963,3157,నాటౌట్,NN,O
47964,3157,),SYM,O


In [ ]:
hindi.POS.unique()

array(['NNP', 'NN', 'VM', 'SYM', 'RB', 'QC', 'PRP', 'QF', 'PSP', 'CC',
       'NST', 'JJ', 'DEM', 'QO', 'RP', 'INTF', 'UT', 'NNP\u200c', 'CL',
       'WQ', 'INJ', 'RDP', 'NNO', 'O', 'SYMP', '_ఆ'], dtype=object)

In [ ]:
hindi.TAG.unique()

array(['B-LOC', 'O', 'B-PERSON', 'B-PERIOD', 'I-PERIOD', 'B-ORG',
       'I-PERSON', 'I-ORG', 'B-NUM', 'B-MONEY', 'I-MONEY', 'I-LOC',
       'B-YEAR', 'B-DAY', 'I-NUM', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME'],
      dtype=object)

In [ ]:
hindi['Sentence'].nunique(), hindi.word.nunique(), hindi.POS.nunique(), hindi.TAG.nunique()

(3157, 15740, 26, 19)

In [ ]:

hindi.TAG.value_counts()

O           39255
B-LOC        1915
B-PERSON     1563
B-NUM         951
I-PERSON      850
B-ORG         778
I-ORG         678
I-MONEY       296
B-DATE        275
B-PERIOD      255
B-MONEY       188
B-DAY         179
I-NUM         168
I-DATE        166
I-PERIOD      127
I-LOC         106
B-YEAR         93
B-TIME         63
I-TIME         60
Name: TAG, dtype: int64

In [ ]:
# parsing the data


In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# hindi_train, hindi_test = train_test_split( hindi, test_size=0.33, random_state=42)

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["TAG"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(hindi)

In [ ]:
sentences = [[s[0] for s in sent] for sent in getter.sentences]
sentences[0]

['ఢిల్లీ', 'పీఠంపై', 'కేజ్రీవాల్\u200c', 'కన్ను', 'పడింది', '.']

In [ ]:
poses = [[s[1] for s in sent] for sent in getter.sentences]
print(poses[0])

['NNP', 'NN', 'NNP', 'NN', 'VM', 'SYM']


In [ ]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])

['B-LOC', 'O', 'B-PERSON', 'O', 'O', 'O']


In [ ]:
# Make TAG name into index for training
tags_vals = list(set(hindi["TAG"].values))
tags_vals.append('X')
tags_vals.append('[CLS]')
tags_vals.append('[SEP]')
tags_vals = set(tags_vals)
tags_vals

{'B-DATE',
 'B-DAY',
 'B-LOC',
 'B-MONEY',
 'B-NUM',
 'B-ORG',
 'B-PERIOD',
 'B-PERSON',
 'B-TIME',
 'B-YEAR',
 'I-DATE',
 'I-LOC',
 'I-MONEY',
 'I-NUM',
 'I-ORG',
 'I-PERIOD',
 'I-PERSON',
 'I-TIME',
 'O',
 'X',
 '[CLS]',
 '[SEP]'}

In [ ]:
tag2idx={'B-DATE': 1,
 'B-DAY':2,
 'B-LOC':3,
 'B-MONEY':4,
 'B-NUM':5,
 'B-ORG':6,
 'B-PERIOD':7,
 'B-PERSON':8,
 'B-TIME':9,
 'B-YEAR':10,
 'I-DATE':11,
 'I-LOC':12,
 'I-MONEY':13,
 'I-NUM':14,
 'I-ORG':15,
 'I-PERIOD':16,
 'I-PERSON':17,
 'I-TIME':18,
 'O':19,
 'X':20,
 '[CLS]':21,
 '[SEP]':22
 }
tag2idx

{'B-DATE': 1,
 'B-DAY': 2,
 'B-LOC': 3,
 'B-MONEY': 4,
 'B-NUM': 5,
 'B-ORG': 6,
 'B-PERIOD': 7,
 'B-PERSON': 8,
 'B-TIME': 9,
 'B-YEAR': 10,
 'I-DATE': 11,
 'I-LOC': 12,
 'I-MONEY': 13,
 'I-NUM': 14,
 'I-ORG': 15,
 'I-PERIOD': 16,
 'I-PERSON': 17,
 'I-TIME': 18,
 'O': 19,
 'X': 20,
 '[CLS]': 21,
 '[SEP]': 22}

In [ ]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [ ]:
# training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

0

In [ ]:
# vocabulary = "/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased/bert-base-multilingual-cased-vocab.txt"

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
max_len = 45 
# tokenizer = AutoTokenizer.from_pretrained("skillzzzzzy/telberto")

In [ ]:



tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
# model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1

No.0,len:24
texts:[CLS] ఢ ##ి ##ల్ ##లీ ప ##ీ ##ఠ ##ం ##పై క ##ే ##జ్ ##రీ ##వ ##ాల్ క ##న్న ##ు ప ##డి ##ంది . [SEP]
No.0,len:24
lables:[CLS] B-LOC X X X O X X X X B-PERSON X X X X X O X X O X X O [SEP]
No.1,len:66
texts:[CLS] మ ##ు ##ఖ ##్య ##మం ##త్రి ప ##ద ##వి ##లో మ ##ు ##చ్ ##చ ##ట ##గా మూడు న ##ె ##ల ##లు ఉ ##ండి ట ##ా ##టా బ ##ై ##బ ##ై చ ##ె ##ప్ ##పి ##న ఆయన మ ##రో మ ##ారు స ##ర్ ##క ##ారు ఏ ##ర్ ##పాటు ది ##శ ##గా ఆ ##లో ##చన ##లు చ ##ే ##స్ ##తున్న ##ట్లు స ##మా ##చ ##ారం . [SEP]
No.1,len:66
lables:[CLS] O X X X X X O X X X O X X X X X B-PERIOD I-PERIOD X X X O X O X X O X X X O X X X X O O X O X O X X X O X X O X X O X X X O X X X X O X X X O [SEP]
No.2,len:62
texts:[CLS] అ ##ందుకు క ##ా ##ంగ్ ##రెస్ తో క ##లి ##సి చ ##ెట్ ##టా ##ప ##ట్ ##టా ##ల్ వ ##ే ##య ##డానికి కూడా వ ##ె ##ను ##క ##ా ##డ ##బ ##ో ##న ##ని అ ##ర ##వి ##ంద ##్ క ##ే ##జ్ ##రీ ##వ ##ాల్ స ##ం ##క ##ేత ##ాలు ప ##ం ##పి ##ం ##చ ##టం వి ##శ ##ే ##ష ##ం . [SEP]
No.2,len:62
lables:[CLS] O X B-ORG X X X O O X 

In [ ]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[   101   1213  15868  15959  54277   1220  18564 111334  13652  59041
   1201  17729  43020  40686  18892  62385   1201  37218  15697   1220
  22012  19052    119    102      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0]


In [ ]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])


[21  3 20 20 20 19 20 20 20 20  8 20 20 20 20 20 19 20 20 19 20 20 19 22
 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19]


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.3)

In [ ]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(2209, 948, 2209, 948)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [ ]:
batch_num = 32

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [ ]:
# model_file_address = '/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased'
# max_len  = 45
# model = AutoModelForMaskedLM.from_pretrained("skillzzzzzy/telberto")
model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased",num_labels=len(tag2idx))

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
m_labels = len(tag2idx)

In [ ]:
model;

In [ ]:
# model.cuda();

In [ ]:
epochs = 20
max_grad_norm = 1.0

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [ ]:
# fine tuning
FULL_FINETUNING = True

In [ ]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
model.train();

In [ ]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

***** Running training *****
  Num examples = 2209
  Batch size = 32
  Num steps = 1400


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
bert_out_address = 'ashiNLP/bert_telberto_model/telegu'

In [ ]:
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)

In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model

In [ ]:
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

In [ ]:
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

('ashiNLP/bert_telberto_model/telegu/vocab.json',
 'ashiNLP/bert_telberto_model/telegu/merges.txt')

In [ ]:
# load model
model = AutoModelForMaskedLM.from_pretrained(bert_out_address,num_labels=len(tag2idx))

In [ ]:
model.cuda();

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
# EVALUATION

In [ ]:
model.eval();

In [ ]:

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
# output_eval_file = os.path.join('/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased', "eval_results.txt")
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =948
  Batch size = 32


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1 socre: 0.684531
Accuracy score: 0.890381
***** Eval results *****

              precision    recall  f1-score   support

        DATE     0.8939    0.9219    0.9077        64
         DAY     0.9787    0.9583    0.9684        48
         LOC     0.8061    0.7670    0.7861       412
       MONEY     0.4839    0.4839    0.4839        31
         NUM     0.7449    0.7978    0.7704       183
         ORG     0.4892    0.5312    0.5094       128
      PERIOD     0.6667    0.6780    0.6723        59
      PERSON     0.5265    0.5099    0.5181       253
        SEP]     0.0000    0.0000    0.0000         0
        TIME     0.7059    0.6000    0.6486        20
        YEAR     0.8571    0.6000    0.7059        20
           _     0.0000    0.0000    0.0000         0

   micro avg     0.6771    0.6921    0.6845      1218
   macro avg     0.5961    0.5707    0.5809      1218
weighted avg     0.7012    0.6921    0.6958      1218

f1 socre: 0.684531
Accuracy score: 0.890381


In [ ]:
save_model_address = 'ashiNLP/bert_telberto_model/telegu'

In [ ]:
save_model = AutoModelForMaskedLM.from_pretrained(save_model_address,num_labels=len(tag2idx))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_model_address,do_lower_case=False)

In [ ]:
# predict
test_query = "20 ఏళ్లుగా ఢిల్లీలో ఉంటున్నాను"

In [ ]:
tokenized_texts = []

temp_token = []
temp_token.append('[CLS]')
token_list = tokenizer.tokenize(test_query)

token_list

['20',
 'Ġà°ıà°³',
 'à±į',
 'à°²',
 'à±ģ',
 'à°Ĺ',
 'à°¾',
 'Ġà°¢',
 'à°¿',
 'à°²',
 'à±į',
 'à°²',
 'à±Ģ',
 'à°²',
 'à±ĭ',
 'Ġà°ī',
 'à°Ĥ',
 'à°Ł',
 'à±ģ',
 'à°¨',
 'à±į',
 'à°¨',
 'à°¾',
 'à°¨',
 'à±ģ']

In [ ]:
for m,token in enumerate(token_list):
    temp_token.append(token)

In [ ]:
if len(temp_token) > max_len-1:
    temp_token= temp_token[:max_len-1]

In [ ]:
temp_token.append('[SEP]')
temp_token

['[CLS]',
 '20',
 'Ġà°ıà°³',
 'à±į',
 'à°²',
 'à±ģ',
 'à°Ĺ',
 'à°¾',
 'Ġà°¢',
 'à°¿',
 'à°²',
 'à±į',
 'à°²',
 'à±Ģ',
 'à°²',
 'à±ĭ',
 'Ġà°ī',
 'à°Ĥ',
 'à°Ł',
 'à±ģ',
 'à°¨',
 'à±į',
 'à°¨',
 'à°¾',
 'à°¨',
 'à±ģ',
 '[SEP]']

In [ ]:
tokenized_texts.append(temp_token)

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])


[  3 851 839 264 271 267 284 266 751 265 271 264 271 285 271 277 316 270
 276 267 268 264 268 266 268 267   3   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:

segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];


In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
segment_ids = torch.tensor(segment_ids)

In [ ]:
model.eval();

In [ ]:
with torch.no_grad():
        outputs = save_model(input_ids, token_type_ids=None,
        attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 

In [ ]:
predict_results = logits.detach().cpu().numpy()
predict_results.shape


(1, 45, 30522)

In [ ]:
from scipy.special import softmax
result_arrays_soft = softmax(predict_results[0])
result_arrays_soft[0]


array([2.2554092e-08, 1.3608058e-11, 9.7245038e-11, ..., 1.8301392e-13,
       2.5563988e-13, 1.4457036e-13], dtype=float32)

In [ ]:
result_array = result_arrays_soft
len(result_array),len(result_array[0])

(45, 30522)

In [ ]:
# Get each token final predict tag index result
result_list = np.argmax(result_array,axis=-1)
result_list

array([ 21,  20,  16,  20,  20,  20,  20,  20, 751,  20,  20,  20,  20,
        20,  20,  20,  19,  20,  20,  20,  20,  20,  20,  20,  20,  20,
        22,  19,  19,  19,  19,  19,  19,  19,  21,  19,  21,  21,  21,
        21,  21,  21,  19,  19,  19])

In [ ]:
for i, mark in enumerate(attention_masks[0]):
    if mark>0:
        print("Token:%s"%(temp_token[i]))
#         print("Tag:%s"%(result_list[i]))
        print("Predict_Tag:%s"%(tag2name[result_list[i]]))
        #print("Posibility:%f"%(result_array[i][result_list[i]]))
        print()

Token:[CLS]
Predict_Tag:[CLS]

Token:20
Predict_Tag:X

Token:Ġà°ıà°³
Predict_Tag:I-PERIOD

Token:à±į
Predict_Tag:X

Token:à°²
Predict_Tag:X

Token:à±ģ
Predict_Tag:X

Token:à°Ĺ
Predict_Tag:X

Token:à°¾
Predict_Tag:X

Token:Ġà°¢


KeyError: ignored